In [1]:
# As before...
import numpy as np
import scipy as sp
import pandas as pd

nat = 195               # Number of atoms
a = 12.55               # Cell size

# Load our previous work

HDF5's are easy to load

In [2]:
xyz = pd.read_hdf('xyz.hdf5', 'xyz')
xyz.head()

symbol          x          y         z
frame atom                                       
0     0        Na   4.536365   7.754558  5.098497
      1         O   9.969693  11.411853  3.889800
      2         O   7.297978  14.646000  1.039326
      3         O  12.633737   2.164139  8.549592
      4         O   7.996152   5.646017  4.587208

# Computing atom to atom distances

This is arguably the most difficult part of this tutorial. How do we account for periodicity?

Lets start by considering **free boundary** conditions first!

Computing all atom to atom distances (**per frame**) increases factorially,

\begin{equation}
    \frac{nat!}{2!\left(nat - 2\right)!} \left(= \frac{1}{2}\left(nat * \left(nat - 1\right)\right)\right)
\end{equation}

in computations (where *nat* is the number of atoms). Fortunately for us, computing the distances can be 
passed off to scipy's [pdist](https://scipy.github.io/devdocs/generated/scipy.spatial.distance.pdist.html).

**CODING TIME: Write a function to compute all of the atom to atom distances in every frame assuming free boundary conditions**

In [3]:
from scipy.spatial.distance import pdist


def skeleton_free_boundary_distances(frame):    # Note that this is frame, not DataFrame
    '''
    Compute all of the atom to atom distances with free boundary conditions
    '''
    # Compute distances
    xyz = frame.loc[:, ['x', 'y', 'z']]
    distances = pdist(xyz)
    # Compute the symbols
    #
    symbols = None
    return pd.DataFrame.from_dict({'distances': distances, 'symbols': symbols})

In [4]:
twobody = xyz.groupby(level='frame').apply(skeleton_free_boundary_distances)
twobody.head()

distances symbols
frame                     
0     0   6.660165     NaN
      1   8.461403     NaN
      2  10.427381     NaN
      3   4.083808     NaN
      4  12.748121     NaN

Distances are no good to us, unless we know where they came from (or at least what two symbols they represent)...

**HINT: Checkout the "combinations" function in the itertools library (part of the Python standard library)**

In [5]:
from itertools import combinations

In [7]:
# %load -s free_boundary_distances, snippets/distances.py
def free_boundary_distances(frame):
    '''
    Compute all of the atom to atom distances with free boundary conditions
    '''
    xyz = frame.loc[:, ['x', 'y', 'z']]
    distances = pdist(xyz)                                          # Compute distances
    symbol = frame.loc[:, 'symbol']
    symbols = [''.join(syms) for syms in combinations(symbol, 2)]   # Compute symbols
    return pd.DataFrame.from_dict({'distances': distances, 'symbols': symbols})


In [8]:
twobody = xyz.groupby(level='frame').apply(free_boundary_distances)
twobody.head()

distances symbols
frame                     
0     0   6.660165     NaO
      1   8.461403     NaO
      2  10.427381     NaO
      3   4.083808     NaO
      4  12.748121     NaO

# Again lets test...

In [9]:
twobody.loc[0].head()

,distances,symbols
0,6.660165,NaO
1,8.461403,NaO
2,10.427381,NaO
3,4.083808,NaO
4,12.748121,NaO


In [10]:
first = xyz.loc[0, ['x', 'y', 'z']].values
for i in range(1, 6):
    print(((first[0, :] - first[i, :])**2).sum()**0.5)

6.6601647141
8.46140284812
10.4273806451
4.08380801487
12.7481208103


# Periodicity

That was fun but it doesn't do what we need it to! Periodic boundaries can be handled a number of ways, here is one algorithm:

- Put all atoms back in unit cell
- Generate a 3x3x3 supercell from the unit cell
- Compute distances looking only from the central unit cell (the internal cell that is completely surrounded by replicas)

Since this is complicated, we will walk through the pieces of the code individually (applying them to a single frame) before putting it all together.

In [11]:
frame = xyz.loc[0]

**CODING TIME: Put all atoms back into the unit cell**

In [12]:
def skeleton_unitify(df, a):
    '''
    Put all atoms back into the cubic unit cell
    '''
    pass

In [13]:
unit_frame = skeleton_unitify(frame, a)    # a is defined above
unit_frame

The % (modulo) operator is nice for such tasks...

In [16]:
# %load -s unitify, snippets/distances.py
def unitify(df, a):
    '''
    Put all atoms back into the cubic unit cell.
    '''
    unitdf = df.copy()              # Don't alter the original
    cell_dim = np.array([a, a, a])
    unitdf.loc[:, ['x', 'y', 'z']] = np.mod(unitdf.loc[:, ['x', 'y', 'z']], cell_dim)
    return unitdf


In [17]:
unit_frame = unitify(frame, a)
unit_frame.shape

(195, 4)

In [18]:
print(unit_frame is not frame)                          # True if objects are same in memory
print(np.all(unit_frame.values == frame.values))        # True if objects' xyz positions are identical

True
False


**CODING TIME: Generate the 3x3x3 superframe from the unit_frame**

In [19]:
def skeleton_superframe(frame, a):
    '''
    Generate a 3x3x3 supercell of a given frame.
    '''
    v = [-1, 0, 1]
    n = len(frame)
    unit = frame.loc[:, ['x', 'y', 'z']].values
    coords = np.empty((n * 27, 3))
    h = 0
    for i in v:
        for j in v:
            for k in v:
                pass
                #for ...
    return coords

In [20]:
big_frame = skeleton_superframe(unit_frame, a)
big_frame.shape

(5265, 3)

One solution...or if you want to have some [fun](#fun)...

In [27]:
# %load -s superframe, snippets/distances.py
def superframe(frame, a):
    '''
    Generate a 3x3x3 supercell from a frame.
    '''
    v = [-1, 0, 1]
    n = len(frame)
    unit = frame.loc[:, ['x', 'y', 'z']].values
    coords = np.empty((n * 27, 3))
    h = 0
    for i in v:
        for j in v:
            for k in v:
                for l in range(n):
                    coords[h, 0] = unit[l, 0] + i * a
                    coords[h, 1] = unit[l, 1] + j * a
                    coords[h, 2] = unit[l, 2] + k * a
                    h += 1
    return coords


In [38]:
big_frame = superframe(unit_frame, a)
big_frame

array([[ -8.01363456,  -4.7954419 ,  -7.45150329],
       [ -2.5803073 ,  -1.1381475 ,  -8.66019952],
       [ -5.2520218 , -10.4540002 , -11.5106737 ],
       ..., 
       [ 20.7455325 ,  22.5531784 ,  20.2676339 ],
       [ 19.9466367 ,  21.1829626 ,  20.3166193 ],
       [ 18.6081609 ,  17.40274447,  18.3286902 ]])

# KDTree

For a single frame's supercell (to start with) we need to compute the distances from the central frame. 

Lets use a nice feature of scipy/scikit-learn (and of course the mathematicians 
that developed it): the [KDTree](https://scipy.github.io/devdocs/generated/scipy.spatial.KDTree.html#scipy.spatial.KDTree)

See also: [wiki](https://en.wikipedia.org/wiki/K-d_tree)

We are going to use the [Cythonized](http://cython.org/) version of the KDTree implementation.

In [39]:
from scipy.spatial import cKDTree

In [48]:
kd = cKDTree(big_frame)
k = 194                          # Number of distances to compute 
distances, indexes = kd.query(unit_frame.loc[:, ['x', 'y', 'z']], k=k)
distances.shape

(195, 194)

In [49]:
indexes.shape

(195, 194)

We have the distances but we need to shape them into a DataFrame and figure out what symbol pair 
each distance belongs too (that last part is critical for the third task).

The first column in the indexes are the indexes of the source atom from which we are looking.

The rest of the columns contain the indexes of the paired atom to which we are computing the distances.

We map superframe indexes back onto the unit_frame indexes:

In [50]:
def map_x_to_y(x, y):
    '''
    Using the indexes in x, generate an array of the same 
    length populated by values from y.
    '''
    mapped = np.empty((len(x), ), dtype=np.int)
    for i, index in enumerate(x):
        mapped[i] = y[index]
    return mapped

In [51]:
unit_frame_indexes = unit_frame.index.get_level_values('atom').tolist() * 27
repeated_source = np.repeat(indexes[:, 0], k)
atom1_indexes = pd.Series(map_x_to_y(repeated_source, unit_frame_indexes))
atom2_indexes = pd.Series(map_x_to_y(indexes.flatten(), unit_frame_indexes))

Now lets convert these Series (a pandas Series is simply a column in a DataFrame)
to symbols using the **map** function.

In [52]:
symbols = unit_frame['symbol'].to_dict()

In [53]:
atom1_symbols = atom1_indexes.map(symbols)
atom2_symbols = atom2_indexes.map(symbols)

In [54]:
symbols = [''.join((first, atom2_symbols[i])) for i, first in enumerate(atom1_symbols)]

Now lets finish this by generating our (periodic) two body DataFrame for this first frame

**CODING TIME: Build the twobody DataFrame for this first frame and prune it to remove 0 length distances.**

In [55]:
frame_twobody = pd.DataFrame.from_dict({'distance': distances.flatten(),
                                        'symbols': symbols})

min_distance = 0.3
frame_twobody = frame_twobody.loc[frame_twobody['distance'] > min_distance]
frame_twobody.head()

,distance,symbols
1,2.256660,NaO
2,2.319464,NaO
3,2.420869,NaO
4,2.428699,NaO
5,2.509244,NaH


We should probably do a check here, but in the interest of time, and because,
with our current implementation this is not trivial, lets just skip this...

# Putting the pieces together

Though we have done this for a single frame, lets see if we can combine all of the pieces 
to act on the original xyz DataFrame.

In [56]:
from scipy.spatial import cKDTree
from snippets.distances import superframe_numba, map_x_to_y_numba, unitify


def cubic_periodic_distances(xyz, a, nat, k, min_distance=0.3):
    '''
    Computes atom to atom distances for a periodic cubic cell.

    Args:
        xyz: Properly indexed pandas DataFrame
        a: Cubic cell dimension

    Returns:
        twobody: DataFrame of distances
    '''
    # Since the unit cell size doesn't change between frames, 
    # lets put all of the atoms (in every frame) back in the
    # unit cell at the same time.
    unit_xyz = unitify(xyz, a)
    # Now we will define another function which will do the 
    # steps we outlined above (see below) and apply this 
    # function to every frame of the unit_xyz
    twobody = unit_xyz.groupby(level='frame').apply(_compute, k=k, min_distance=min_distance, 
                                                    max_distance=a)
    # Pair the symbols
    twobody.loc[:, 'symbols'] = twobody['atom1'] + twobody['atom2']
    # Name the indexes
    twobody.index.names = ['frame', 'two']
    return twobody


What should the function **_compute** look like/do?

**NOT QUITE CODING TIME: Just load the function**

In [58]:
# %load -s _compute, snippets/distances.py
def _compute(unit_frame, k, min_distance, max_distance=25.0):
    '''
    Compute periodic atom to atom distances
    '''
    # Generate superframe
    values = unit_frame.loc[:, ['x', 'y', 'z']].values
    big_frame = superframe_numba(values, a)

    # Create the K-D tree
    kd = cKDTree(big_frame)
    distances, indexes = kd.query(values, k=k)

    # Metadata
    unit_frame_indexes = np.tile(unit_frame.index.get_level_values('atom').values, (len(values), 27)).flatten()
    symbol_dict = unit_frame.reset_index('frame', drop=True).loc[:, 'symbol'].to_dict()
    repeated_source = np.repeat(indexes[:, 0], k)
    def symbol_caller(symbol):
        return symbol_dict[symbol]

    # Mapping of atom indexes to symbols
    atom1_indexes = map_x_to_y_numba(repeated_source, unit_frame_indexes)
    atom2_indexes = map_x_to_y_numba(indexes.flatten(), unit_frame_indexes)
    atom1_symbols = list(map(symbol_caller, atom1_indexes))
    atom2_symbols = list(map(symbol_caller, atom2_indexes))

    # Generation of the DataFrame
    frame_twobody = pd.DataFrame.from_dict({'distance': distances.flatten(), 'atom1': atom1_symbols, 'atom2': atom2_symbols})
    frame_twobody = frame_twobody.loc[frame_twobody['distance'] > min_distance]
    return frame_twobody


In [59]:
# WARNING: Reduce the value of k else this will take about 2 minutes!
%time twobody = cubic_periodic_distances(xyz, a, nat, k=194)
twobody.head()

Wall time: 1min 51s


atom1 atom2  distance symbols
frame two                              
0     1      Na     O  2.256660     NaO
      2      Na     O  2.319464     NaO
      3      Na     O  2.420869     NaO
      4      Na     O  2.428699     NaO
      5      Na     H  2.509244     NaH

# Saving

Now that we did that heavy analysis (it took alm
lets save our data again.

In [60]:
store = pd.HDFStore('twobody.hdf5', mode='w')
%time store.put('twobody', twobody)
store.close()

Wall time: 26 s


Again, though there are a bunch of improvements/features we could make, lets move on...

# ...on to step [three](03_pcf.ipynb)

<a id='fun'></a>
# Numba-fied fun!

Python has a way to optimize big loops.

This is for learning and fun, remember the first rule of optimization: optimize the slowest step first!

[numba](http://numba.pydata.org/) is a beautiful and powerful way to "just-in-time" compile python code into native machine code...

In [21]:
from numba import jit, float64

In [ ]:
# %load -s superframe, snippets/distances.py
def superframe(frame, a):
    '''
    Generate a 3x3x3 supercell from a frame.
    '''
    v = [-1, 0, 1]
    n = len(frame)
    unit = frame.loc[:, ['x', 'y', 'z']].values
    coords = np.empty((n * 27, 3))
    h = 0
    for i in v:
        for j in v:
            for k in v:
                for l in range(n):
                    coords[h, 0] = unit[l, 0] + i * a
                    coords[h, 1] = unit[l, 1] + j * a
                    coords[h, 2] = unit[l, 2] + k * a
                    h += 1
    return coords


In [22]:
# %load -s superframe_numba, snippets/distances.py
@jit(nopython=True)
def superframe_numba(unit, a):
    v = [-1, 0, 1]
    n = len(unit)
    coords = np.empty((n * 27, 3), dtype=float64)
    h = 0
    for i in v:
        for j in v:
            for k in v:
                for l in range(n):
                    coords[h, 0] = unit[l, 0] + i * a
                    coords[h, 1] = unit[l, 1] + j * a
                    coords[h, 2] = unit[l, 2] + k * a
                    h += 1
    return coords

In [24]:
n = 100
%timeit -n $n superframe(unit_frame, a)
%timeit -n $n superframe_numba(unit_frame.loc[:, ['x', 'y', 'z']].values, a)

100 loops, best of 3: 7.66 ms per loop
100 loops, best of 3: 637 µs per loop


~10x speedup (for 1 line of code)